In [172]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_nomove_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10 # Try values like 20, 50, or 100
n_nonzero_coefs = 5 # Sparsity level

In [173]:
def create_sparse_representation(data, n_components, n_nonzero_coefs):
    """
    Create a sparse representation with reduced dimensions.
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Reduced dimensionality
    - n_nonzero_coefs: Sparsity level
    
    Returns:
    - Sparse representation with reduced dimensions.
    """
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)

        # Learn a dictionary (or use a pre-defined one)
        dict_learner = DictionaryLearning(
            n_components=n_components, 
            transform_algorithm='lasso_lars', 
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        dict_learner.fit(feature_data)
        sparse_codes = dict_learner.transform(feature_data)

        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes

    return sparse_representation

In [174]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))
print(train_data_combined.shape)

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', transform_n_nonzero_coefs=n_nonzero_coefs, n_jobs= -1)
dict_learner.fit(train_data_combined)
# D = dict_learner.components_  # Shape: (n_components, feature_dim)


train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
print(train_sparse_codes.shape)
print(train_sparse_codes[0])


(1600, 251)
(1600, 10)
[ 46321.22552049    653.07467902 -12000.59906377  10762.53533634
   3891.98834828   5856.6171165    1051.53068869  -2240.67061497
   2306.16880072  -1475.8435746 ]


In [175]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
print('Test data shape:', test_sparse_codes[0])



Test data shape: [ 48594.00955596 -16890.20528154  -5584.88521833  -5535.78337742
  -7065.13999764   1404.64733929  -2815.54203095   -311.01695619
   -991.95891109   1089.31182222]


In [176]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [177]:
# Dictionary
D = train_sparse_codes.T
print('Dictionary shape:', D.shape)

Dictionary shape: (10, 1600)


In [178]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=5):
    print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    print(f'Residual: {residual}')
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_sparse_codes:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
print(predictions.shape)


tSample: (10,)
Residual: 1502.999684162175
Residual: 53802.80589946907
tSample: (10,)
Residual: 3403.9897338593937
Residual: 49966.27987024536
tSample: (10,)
Residual: 3161.9442182047
Residual: 50618.43972477912
tSample: (10,)
Residual: 455.40159852151095
Residual: 53105.91967203126
tSample: (10,)
Residual: 1613.463378993477
Residual: 53607.75609513405
tSample: (10,)
Residual: 14512.650976018007
Residual: 43708.864325303664
tSample: (10,)
Residual: 18784.037150820062
Residual: 40180.57256274274
tSample: (10,)
Residual: 1295.745414633447
Residual: 51264.75326760975
tSample: (10,)
Residual: 14728.555905636485
Residual: 43201.51310025594
tSample: (10,)
Residual: 4892.452771128346
Residual: 51869.43258218287
tSample: (10,)
Residual: 4599.047807939133
Residual: 49415.73624007067
tSample: (10,)
Residual: 4326.524737441561
Residual: 54270.6261437985
tSample: (10,)
Residual: 2260.9417308431207
Residual: 50131.14783053733
tSample: (10,)
Residual: 804.0438916629538
Residual: 44505.69059574085
tS

In [179]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 100.00%
tp: 200
tn: 200
fp: 0
fn: 0
MDR: 0.0
FAR: 0.0
AR: 1.0
